Install split_folders app

In [ ]:
pip install split_folders

Import modul, download data, split folder, buat ImageDataGenerator, bandingkan dengan ImageDataGenerator dengan train dan verifikasi image di validation

In [ ]:
#import dan check versi tf
import tensorflow as tf
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
print(tf.__version__)
import sklearn
from sklearn.model_selection import train_test_split
from sklearn import datasets
import zipfile
import os 
 
#download data 
!wget --no-check-certificate \
https://dicodingacademy.blob.core.windows.net/picodiploma/ml_pemula_academy/rockpaperscissors.zip \
-O /tmp/rockpaperscissors.zip

import zipfile
import os
local_zip = '/tmp/rockpaperscissors.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

os.listdir ('/tmp/rockpaperscissors')

#base_dir = '/tmp/rockpaperscissors/rps-cv-images'
#train_dir = os.path.join(base_dir, 'train')
#validation_dir = os.path.join(base_dir, 'val')
#os.listdir('/tmp/rockpaperscissors/rps-cv-images')

import split_folders
base_dir = '/tmp/rockpaperscissors/rps-cv-images'
split_folders.ratio('/tmp/rockpaperscissors/rps-cv-images', output="/tmp/rockpaperscissors/data", seed=1337, ratio=(.6, .4))
#HasilSplitData = '/tmp/rockpaperscissors/data'
rock_dir = os.path.join(base_dir, 'rock')
paper_dir = os.path.join(base_dir, 'paper')
scissors_dir = os.path.join(base_dir, 'scissors')

train_rock_dir, validation_rock_dir = train_test_split(os.listdir (rock_dir))
train_paper_dir, validation_paper_dir = train_test_split(os.listdir (paper_dir))
train_scissors_dir, validation_scissors_dir = train_test_split(os.listdir (scissors_dir))

#validation_rock_dir = os.path.join(HasilSplitData, 'rock')
#validation_paper_dir = os.path.join(HasilSplitData, 'paper')
#validation_scissor_dir = os.path.join(HasilSplitData, 'scissors')


datamodel = ImageDataGenerator(
                        rescale=1./255,
                        rotation_range=20,
                        horizontal_flip=True,
                        shear_range = 0.2,
                        fill_mode = 'wrap')

train_generator = datamodel.flow_from_directory(
        base_dir,  
        target_size=(200, 200),
        batch_size=4,
        class_mode='categorical')

validation_generator = datamodel.flow_from_directory(
        base_dir,
        target_size=(200, 200),
        batch_size=4,
        class_mode='categorical')


configurasi model image

In [ ]:
#membuat model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation = 'relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(256,(3,3), activation = 'relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation = 'relu'),
    tf.keras.layers.Dense(3, activation = 'softmax')
])

#model bisa belajar
model.compile(loss = 'categorical_crossentropy',
                optimizer = tf.optimizers.Adam(),
                metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch = 25,
    epochs = 20,
    validation_data = validation_generator,
    validation_steps = 5,
    verbose = 2)

#upload file image
#from google.colab import files
#uploaded = files.upload()
#for fn in uploaded.keys():
 # print('User uploaded file "{name}" with length {length} bytes'.format(
  #    name=fn, length=len(uploaded[fn])))

Klik run untuk masukan file image yang akan di analisa machine-learning

In [ ]:
import numpy as np
from google.colab import files
from keras.preprocessing import image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# predicting images     
uploaded = files.upload()
     
for fn in uploaded.keys():     
  path = fn
  img = image.load_img(path, target_size=(200,200))
  imgplot = plt.imshow(img)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
     
  images = np.vstack([x])
  classes = model.predict(images, batch_size=10)
      
  print(fn)
  if classes[0][0] == 1:
    print('paper')
  elif classes[0][1] == 1:
    print('rock')
  elif classes[0][2] == 1:
    print('scissor')